# This script samples images and faces for handcoding 

In [1]:
from facelib import FaceDetector, AgeGenderEstimator

In [2]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.patches as patches
import os
import random

# importing pandas as pd  
import pandas as pd  
from IPython.display import clear_output
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mplt

# For this scrip to run, you will need to request acess to the raw data images. you then need to place the posts folder in your working directory

# I make an input loop to create the hand coded labels. 
It takes 1000 random images, and outputs 1 random face from that image - this image i then code 

In [3]:
random.seed(420)

In [4]:
path = "posts"

paths_list = []


i = 0
while i < 1000:
    #draw random politician
    int_1 = random.randint(0,len(os.listdir(path))-1)
    
    #create a path into the folder
    impath = path + "\\"+ os.listdir(path)[int_1]+"\\" 
    
    #draw a random image in that folder (the first 4 items are meta data)
    int_2 = random.randint(4,len(os.listdir(impath))-1)
    
    #create path to selected image
    impath = impath + os.listdir(impath)[int_2]
    
    #if we by some change have gotten something other than an image
    if impath[-4:] != "jpeg":
        continue
    
    #if the image is not already in the drawn images - save the im path
    if impath not in paths_list:
        paths_list.append(impath)
        i +=1
    else:
        continue
    

Predict the amount of people in the images, and save the images with faces and boxes around for manual labelling

In [5]:
gender_dict = {'Female' : 'Woman',
               'Male'   : 'Man'}

In [6]:
face_detector = FaceDetector(confidence_threshold=0.7)
age_gender_detector = AgeGenderEstimator()

from FaceDetector: weights loaded
from AgeGenderEstimator: weights loaded


In [7]:
face_list = []
gender_list = []
boxes_list = []
    #try:
    #    image = mplt.imread(impath)
    #except:
    #    continue
ind = 0    
n_people_list = []

for p in tqdm(paths_list):
    
    #detect faces
    image = mplt.imread(p)
    image = np.array(image)

    faces, boxes, scores, landmarks = face_detector.detect_align(image)
    if len(faces) == 0:
        n_people_list.append(0)
        face_list.append([faces])
        gender_list.append([faces]) #just empty tensor
        boxes_list.append([boxes])
        
        img_pil = Image.fromarray(np.array(image))
        img = ImageDraw.Draw(img_pil) 
        
        img_pil.save("handcode\\all_people\\" + str(ind) + ".jpeg")



    else:
        n_people_list.append(len(faces))
        genders, ages = age_gender_detector.detect(faces)


        img_pil = Image.fromarray(np.array(image))
        img = ImageDraw.Draw(img_pil) 
        
        #append faces - should not be done for manual labelling to avoid bias
        #for i, f in enumerate(faces):
         #   img.rectangle(np.array(boxes)[i],outline= "red")
          #  img.text(xy = (np.array(boxes)[i][0],np.array(boxes)[i][1] + 15),
           #          text = gender_dict[genders[i]],
            #        fill = "white")

        face_list.append([faces])
        gender_list.append([genders])
        boxes_list.append([boxes])
        img_pil.save("handcode\\all_people\\" + str(ind) + ".jpeg")
        #display(img_pil)
    ind += 1

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [08:32<00:00,  1.95it/s]


I then sample 1000 faces from the found faces and label them

In [8]:
n_people = 0

for f in face_list:
    n_people += len(f[0])
n_people

1207

In [9]:
faces_to_keep = random.sample(range(n_people), 1000)

In [10]:
number = 0
keep = 0
auto_gender = []
img_name_face = []
face_for_df = []


for i, faces in enumerate(boxes_list):
    if len(faces[0]) == 0:
        continue
    else:
        for f, face in enumerate(faces[0]):
            
            if keep in faces_to_keep: #if one of the 1000 sampled faces
                
                
                image = mplt.imread("handcode\\all_people\\" + str(i) + ".jpeg")
                image = np.array(image)
                img_pil = Image.fromarray(np.array(image))
                img = ImageDraw.Draw(img_pil) 

                #draw the face on the images
                img.rectangle(np.array(face),outline= "red", width=3) #draw the face
                
                face_for_df.append(face)

                auto_gender.append(gender_dict[gender_list[i][0][f]]) #save the gender
                img_name_face.append(paths_list[i])

                str(i)+ "_" + str(f)
                #img_pil.save("handcode\\random_face\\" + str(i)+ "_" + str(f) + ".jpeg") #save the image with 1 rectange
                img_pil.save("handcode\\random_face\\" + str(number) + ".jpeg") #save the image with 1 rectange
                keep += 1
                number += 1
            else:
                keep +=1
                continue
            
    print(str(i +1) + " out of 1000 images", end="\r")
    
   

In [11]:
handcode_dict_npeople = {'random_image_number': list(range(1000)) ,
                        'image_name' : paths_list,
                         'auto_people' : n_people_list
                         }

handcode_dict_gender = {'random_face_number' : list(range(1000)),
                        'image_name' :  img_name_face,
                        'face_tensor' : face_for_df,
                         'auto_gender' :  auto_gender}

                 
                 

In [12]:
df1 = pd.DataFrame(handcode_dict_npeople) 
df2 = pd.DataFrame(handcode_dict_gender) 

df1.to_csv('handcode\\n_people_autocode.csv') 
df2.to_csv('handcode\\gender_autocode.csv') 
